# Debiasing Imputation #

This notebook is about dealing with missing data that does not increase bias (gender bias, race bias, etc.), or even potentially reduce it



## Problem statement ##
Most common way to handle missing data is to drop them. The second most common way is to replace the missing data with the most likely value. For the categorical features it is the most frequent value. For the numerical features it is the mean. `scikit-learn` has a class available for this: [SimpleImputer](http://scikit-learn.org/dev/modules/generated/sklearn.impute.SimpleImputer.html). The problem with this approach is that even though it preserves mean, but it reduces the standard deviation, sometimes very significantly. To demonstrate this, let's consider a simple array, then remove half of the values and replace them with mean, and see what happens with STD:

In [1]:
import numpy as np
from scipy.stats import norm, multinomial
original_data = norm.rvs(loc=1.0, scale=0.5, size=1000, random_state=1386)
original_data[:20]

array([1.53547966, 0.99260019, 0.88633099, 1.21320929, 1.03287069,
       1.34151072, 0.98476757, 1.17019719, 1.10089714, 0.48023982,
       1.49781353, 1.21862054, 1.91732282, 0.55931941, 0.53091708,
       1.3266663 , 0.94301855, 1.1107632 , 0.42426201, 1.39311814])

In [2]:
#Now replace every other element with the mean 1.0
missing_elements = np.asarray([0,1]*500)
updated_data = original_data * (1-missing_elements) + missing_elements
updated_data[:20]

array([1.53547966, 1.        , 0.88633099, 1.        , 1.03287069,
       1.        , 0.98476757, 1.        , 1.10089714, 1.        ,
       1.49781353, 1.        , 1.91732282, 1.        , 0.53091708,
       1.        , 0.94301855, 1.        , 0.42426201, 1.        ])

In [3]:
#Now, let's get mean and std of the new distribution:
mean, std = norm.fit(updated_data)
print(f'Mean: {mean}, std: {std}')

Mean: 1.0117580053066189, std: 0.33428315977079176


As you see, even though the mean is the same, the standard deviation is much less. While the imputation of data this way increases the performance of the model, it also amplifies the bias that already exists in the data. In order to prevent amplification of the bias, we have to replace the missing values with a sample from the normal distribution with the same mean and standard deviation. For categorical features it would be a multinomial distribution.

For debiasing we can try to increase the standard deviation of the distribution from which we sample data for numerical features, and a similar transformation for the multinomial distribution. 

In this notebook I suggest two classes for the numerical and categorical features respectively.

## Proposed solution ##

In [161]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy.ma as ma
from sklearn.utils.validation import check_is_fitted
class NumericalUnbiasingImputer(BaseEstimator, TransformerMixin):
    """Un-biasing imputation transformer for completing missing values.
        Parameters
        ----------
        std_scaling_factor : number
            We will multiply std by this factor to increase or decrease bias
    """
    def __init__(self, std_scaling_factor=1, random_state=7294):
        self.std_scaling_factor = std_scaling_factor
        self.random_state = random_state

        
    def fit(self, X: np.ndarray, y=None):
        """Fit the imputer on X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            Input data, where ``n_samples`` is the number of samples and
            ``n_features`` is the number of features.
        Returns
        -------
        self : NumericalUnbiasingImputer
        """
        if len(X.shape) < 2:
            X = X.reshape(-1,1)
        mask = np.isnan(X)
        masked_X = ma.masked_array(X, mask=mask)

        mean_masked = np.ma.mean(masked_X, axis=0)
        std_masked = np.ma.std(masked_X, axis=0)
        mean = np.ma.getdata(mean_masked)
        std = np.ma.getdata(std_masked)
        mean[np.ma.getmask(mean_masked)] = np.nan
        std[np.ma.getmask(std_masked)] = np.nan
        self.mean_ = mean
        self.std_ = std * self.std_scaling_factor

        return self
    
     
    def transform(self, X):
        """Impute all missing values in X.
        Parameters
        ----------
        X : {array-like}, shape (n_samples, n_features)
            The input data to complete.
        """
        check_is_fitted(self, ['mean_', 'std_'])

        if len(X.shape) < 2:
            X = X.reshape(-1,1)
        mask = np.isnan(X)
        n_missing = np.sum(mask, axis=0)
        
        def transform_single(index):
            col = X[:,index].copy()
            mask_col = mask[:, index]
            sample = np.asarray(norm.rvs(loc=self.mean_[index], scale=self.std_[index], 
                                         size=col.shape[0], random_state=self.random_state))
            col[mask_col] = sample[mask_col]
            return col
            
        
        Xnew = np.vstack([transform_single(index) for index,_ in enumerate(n_missing)]).T
        

        return Xnew
    


In [162]:
imputer = NumericalUnbiasingImputer()
missing_indicator = missing_elements.copy().astype(np.float16)
missing_indicator[missing_indicator == 1] = np.nan
data_with_missing_values = original_data + missing_indicator
data_with_missing_values = np.vstack([data_with_missing_values, original_data*5]).T
imputer.fit(data_with_missing_values)
transformed = imputer.transform(data_with_missing_values)
print(transformed[:20,:])
transformed.shape


[[ 1.53547966  7.6773983 ]
 [ 1.28446105  4.96300096]
 [ 0.88633099  4.43165496]
 [ 1.26161414  6.06604643]
 [ 1.03287069  5.16435346]
 [ 1.54999452  6.70755359]
 [ 0.98476757  4.92383783]
 [ 1.78611804  5.85098593]
 [ 1.10089714  5.50448571]
 [ 1.01113317  2.4011991 ]
 [ 1.49781353  7.48906767]
 [ 1.2508774   6.09310269]
 [ 1.91732282  9.58661411]
 [ 1.13145139  2.79659703]
 [ 0.53091708  2.65458541]
 [ 1.81200067  6.6333315 ]
 [ 0.94301855  4.71509276]
 [-0.0711673   5.55381598]
 [ 0.42426201  2.12131004]
 [ 1.94678859  6.96559068]]


(1000, 2)

In [6]:
#Let's see how it is different from the original array:
new_mean, new_std = norm.fit(transformed[:,0])
print(f'Mean: {new_mean}, Std: {new_std}')

Mean: 1.0197348250784546, Std: 0.4659586665233841


Some difference in the standard deviation can be explained, because we fitted the model on the incomplete data.

Now we need to do the same for the categorical features

In the multinomial distribution there is no single parameter responsible for standard deviation. However we can observe, that scaling the standard deviation of the normal distribution is equivalent to scaling `x`. If we do a similar transformation in the multinomial distribution, this would be equivalent to raising the parameters to the power of $\frac{1}{s}$, where $s$ is the scaling factor

In [163]:
import pandas as pd
class CategoricalUnbiasingImputer(BaseEstimator, TransformerMixin):
    """Un-biasing imputation transformer for completing missing values.
        Parameters
        ----------
        std_scaling_factor : number
            We will multiply std by this factor to increase or decrease bias
    """
    def __init__(self, scaling_factor=1, random_state=7294):
        self.scaling_factor = scaling_factor
        self.random_state = random_state

        
    def fit(self, X: np.ndarray, y=None):
        """Fit the imputer on X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            Input data, where ``n_samples`` is the number of samples and
            ``n_features`` is the number of features.
        Returns
        -------
        self : NumericalUnbiasingImputer
        """
        if len(X.shape) < 2:
            X = X.reshape(-1,1)

        def fit_column(column):
            mask = pd.isnull(column)
            column = column[~mask]
            unique_values, counts = np.unique(column.data, return_counts=True)
            total = sum(counts)
            probabilities = np.array([(count/total)**(1/self.scaling_factor) 
                    for count in counts])
            total_probability = sum(probabilities)
            probabilities /= total_probability
            return unique_values, probabilities


        self.statistics_ = [fit_column(X[:,column]) for column in range(X.shape[1])]

        return self
    
     
    def transform(self, X):
        """Impute all missing values in X.
        Parameters
        ----------
        X : {array-like}, shape (n_samples, n_features)
            The input data to complete.
        """
        check_is_fitted(self, ['statistics_'])

        if len(X.shape) < 2:
            X = X.reshape(-1,1)
        
        def transform_single(index):
            column = X[:,index].copy()
            mask = pd.isnull(column)
            values, probabilities = self.statistics_[index]

            sample = np.argmax(multinomial.rvs(p=probabilities, n=1,
                                         size=mask.sum(), random_state=self.random_state), axis=1)
            column[mask] = np.vectorize(lambda pick: values[pick])(sample);
            return column
            
        
        Xnew = np.vstack([transform_single(index) for index in range(len(self.statistics_))]).T
        

        return Xnew

In [164]:
names = np.array(['one', None, 'two', 'three', 'four', 'one', None, 'one', 'two'])
names = names.reshape(-1,1)
cat_imp = CategoricalUnbiasingImputer(random_state=121)
cat_imp.fit(names)
print(cat_imp.statistics_)
imputed = cat_imp.transform(names)
imputed

[(array(['four', 'one', 'three', 'two'], dtype=object), array([0.14285714, 0.42857143, 0.14285714, 0.28571429]))]


array([['one'],
       ['one'],
       ['two'],
       ['three'],
       ['four'],
       ['one'],
       ['one'],
       ['one'],
       ['two']], dtype=object)

Now we test our utilities on the scikit-learn datasets. Let's try our approach on the famous titanic dataset. Let's see if any of the columns contain nans

In [165]:
titanic = pd.read_csv("data/titanic.csv")
titanic.isna().sum(axis=0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [166]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


We see that Age is numeric, and Cabin is an object feature. Let's update them using our imputers

In [167]:
n_imputer = NumericalUnbiasingImputer()
titanic.Age = n_imputer.fit(titanic.Age).transform(titanic.Age)

C:\Users\michael\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\michael\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [168]:
c_imputer = CategoricalUnbiasingImputer()
titanic.Cabin = c_imputer.fit(titanic.Cabin).transform(titanic.Cabin)

C:\Users\michael\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\michael\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [170]:
#Let's see how it transformed Age
titanic.Age.head(20)

0     22.000000
1     38.000000
2     26.000000
3     35.000000
4     35.000000
5     45.875319
6     54.000000
7      2.000000
8     27.000000
9     14.000000
10     4.000000
11    58.000000
12    20.000000
13    39.000000
14    14.000000
15    55.000000
16     2.000000
17    -3.935337
18    31.000000
19    58.066929
Name: Age, dtype: float64

OK, negative age is a bit too much... But keep in mind that the purpose is not to reconstruct the data, but to avoid amplifying bias in the machine learning model. Let's make sure Age and Cabin now is not null

In [172]:
print(titanic.Age.isnull().sum())
print(titanic.Cabin.isnull().sum())

0
0


In [173]:
#Unique values of the Cabin
titanic.Cabin.unique()

array(['C52', 'C85', 'E36', 'C123', 'D17', 'B80', 'E46', 'C30', 'C87',
       'G6', 'C103', 'D6', 'D', 'B51 B53 B55', 'E12', 'D9', 'D33', 'B49',
       'A26', 'C126', 'D56', 'C111', 'A6', 'C70', 'D10 D12', 'C78',
       'C23 C25 C27', 'F G73', 'F2', 'B50', 'B78', 'B20', 'C47', 'D20',
       'C86', 'C93', 'C22 C26', 'C110', 'E38', 'F38', 'B102', 'B35',
       'E121', 'C68', 'D36', 'E8', 'A36', 'B30', 'B94', 'B3', 'C128',
       'B28', 'C83', 'C2', 'F33', 'B96 B98', 'E31', 'C50', 'D26', 'C125',
       'F G63', 'E101', 'D28', 'E50', 'A16', 'F4', 'D35', 'C65', 'A5',
       'A7', 'C124', 'C54', 'B58 B60', 'B77', 'E58', 'E63', 'F E69', 'T',
       'D47', 'A24', 'B86', 'B37', 'E44', 'A31', 'E25', 'B71', 'B18',
       'B101', 'C46', 'E34', 'D49', 'E33', 'C49', 'D45', 'B19', 'C95',
       'B22', 'B5', 'A32', 'C7', 'B4', 'D46', 'D11', 'A23', 'E10', 'E24',
       'E17', 'D15', 'C90', 'C118', 'B69', 'A20', 'B41', 'E67', 'C99',
       'D7', 'E77', 'B42', 'C45', 'C106', 'E49', 'A19', 'C91', 'D19',
 

# Next Steps #

Now we need to find a dataset that is known to have a gender or race bias, and demonstrate, that with this technique we can avoid amplifying bias, and maybe even decrease the bias by applying a scaling factor